# 실습 03: PaddleOCR 기본 사용법

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab03_PaddleOCR.ipynb)

## 🎯 학습 목표
- PaddleOCR 설치 및 기본 사용법
- 한글/영어 혼합 문서 인식
- 결과 시각화

## ⏱️ 소요 시간: 20분
## 📊 난이도: ⭐⭐☆☆☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# PaddleOCR 설치
!pip install -q paddlepaddle paddleocr
print("✅ PaddleOCR 설치 완료!")


In [ ]:
# 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 한글 폰트 로드 (Windows/Linux 호환)
try:
    font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 20)
except:
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 20)
    except:
        font = ImageFont.load_default()

img = Image.new('RGB', (600, 400), color='white')
draw = ImageDraw.Draw(img)

text = """삼성전자 주식회사
Samsung Electronics

품의서 (Purchase Request)

품의번호: PR-2025-0001
작성일자: 2025년 1월 15일
담당부서: 구매팀
"""

draw.text((30, 30), text, fill='black', font=font)
img.save('sample_doc.jpg')

print("✅ 샘플 이미지 생성 완료!")
img


In [ ]:
# PaddleOCR 실행
from paddleocr import PaddleOCR

# 한글+영어 OCR 초기화
ocr = PaddleOCR(use_angle_cls=True, lang='korean')

# OCR 실행 (공식 API)
result = ocr.ocr('sample_doc.jpg', cls=True)

print("\n📄 인식된 텍스트:\n")
print("="*50)

# 결과 출력
# result[0]은 페이지, 각 line은 [박스좌표, (텍스트, 신뢰도)]
for idx, line in enumerate(result[0]):
    text = line[1][0]  # 텍스트
    score = line[1][1]  # 신뢰도
    print(f"{text:30s} (신뢰도: {score:.2%})")

print("="*50)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 한글 폰트 로드 (이미지에 텍스트 표시용)
try:
    label_font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 14)
except:
    try:
        label_font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 14)
    except:
        label_font = ImageFont.load_default()

# 원본 이미지 로드
original_img = cv2.imread('sample_doc.jpg')
original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

print("="*70)
print("🔍 OCR 결과 시각화")
print("="*70)
print(f"원본 이미지 shape: {original_img_rgb.shape} (H, W, C)")

# 이미지에 박스 그리기
pil_img = Image.fromarray(original_img_rgb)
draw = ImageDraw.Draw(pil_img)

print(f"\n📦 인식된 텍스트와 박스 좌표:")

for idx, line in enumerate(result[0]):
    box = line[0]  # 박스 좌표 [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]
    text = line[1][0]  # 텍스트
    score = line[1][1]  # 신뢰도
    
    if score < 0.5:
        continue
    
    # 박스 좌표를 정수로 변환
    points = [(int(point[0]), int(point[1])) for point in box]
    
    # 색상 결정 (높은 신뢰도: 초록, 낮은 신뢰도: 주황)
    color = (0, 255, 0) if score > 0.8 else (255, 165, 0)
    
    # 박스 그리기
    draw.polygon(points, outline=color, width=2)
    
    # 텍스트 레이블 추가
    text_x = int(box[0][0])
    text_y = int(box[0][1]) - 20
    draw.text((text_x, max(0, text_y)), f"{text[:15]} {score:.0%}", fill=color, font=label_font)
    
    # 상세 정보 출력 (처음 3개만)
    if idx < 3:
        print(f"\n[{idx+1}] '{text}' (신뢰도: {score:.1%})")
        print(f"    박스: {points}")

# 결과 시각화
plt.figure(figsize=(15, 10))
plt.imshow(pil_img)
plt.title('PaddleOCR 인식 결과', fontsize=16, fontproperties=fontprop)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"\n✅ 총 {len(result[0])}개의 텍스트 영역 인식 완료!")
print("="*70)